In [ ]:
!pip install -U accelerate
!pip install bitsandbytes==0.42.0

In [ ]:
!pip install -q -U datasets
!pip install transformers==4.36.2 peft==0.9.0
!pip install bitsandbytes==0.42.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch

# Load model and tokenizer
model_id = "microsoft/phi-3-mini-4k-instruct"
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # load model in 4-bit
    bnb_4bit_quant_type="nf4",  # optimal quant type
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)

model.eval()


The repository for microsoft/phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or `pip install bitsandbytes`.

In [ ]:

# # Prompt
# user_input = " How can I find help for an alcohol or drug use proble,"
# prompt = f"<|user|>\n{user_input}\n<|assistant|>\n"

# # Tokenize
# inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# # Define generation config
# generation_config = GenerationConfig(
#     max_new_tokens=200,
#     temperature=0.7,
#     top_p=0.9,
#     do_sample=True,
#     repetition_penalty=1.1
# )

# # Generate output
# with torch.no_grad():
#     outputs = model.generate(
#         **inputs,
#         generation_config=generation_config
#     )

# # Decode and clean output
# decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Extract assistant reply
# if "<|assistant|>" in decoded:
#     bot_reply = decoded.split("<|assistant|>")[-1].strip()
# else:
#     bot_reply = decoded.strip()

# print("🤖 Bot reply (Pre-Finetune):")
# print(bot_reply)


🤖 Bot reply (Pre-Finetune):
How can I find help for an alcohol or drug use proble,
 Finding support and treatment options for substance abuse issues is a crucial step towards recovery. Here are some resources that you might consider reaching out to:

1. Substance Abuse and Mental Health Services Administration (SAMHSA) National Helpline - 1-800-662-HELP (4357), available Monday through Friday from 9 AM to 9 PM ET, offers free, confidential information on substance abuse and mental health disorders in English and Spanish. They also provide referrals to local treatment facilities if needed.

2. Alcoholics Anonymous (AA): AA meetings offer peer support groups specifically designed for individuals struggling with alcohol addiction. These group sessions follow the Twelve Steps program aimed at achieving sobriety by relying upon faith, honesty, integrity, courageous action, willingness to change one's life,


In [ ]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="personal_info.json", split="train", keep_in_memory=True)


# Format into prompt + response pairs
def format_for_chat(example):
    return {
        "text": f"<|user|>\n{example['instruction']}\n<|assistant|>\n{example['response']}"
    }

dataset = dataset.map(format_for_chat)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

In [ ]:
# ✅ Step 4: Apply LoRA for fine-tuning
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType

# Only tune attention projections
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "dense"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

OutOfMemoryError: CUDA out of memory. Tried to allocate 376.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 218.12 MiB is free. Process 48929 has 14.53 GiB memory in use. Of the allocated memory 14.05 GiB is allocated by PyTorch, and 369.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# ✅ Step 5: Tokenize dataset
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_dataset = dataset.map(tokenize)

In [ ]:
# ✅ Step 6: Training config and Trainer setup
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./phi3-mental-finetune",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    learning_rate=2e-4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

In [ ]:
# ✅ Step 7: Save fine-tuned model and tokenizer
model.save_pretrained("phi3-mental-finetune")
tokenizer.save_pretrained("phi3-mental-finetune")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load your fine-tuned model
model_path = "phi3-mental-finetune"  # or the path to your saved model
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# Sample user message
user_input = "I'm feeling very low today. What should I do?"

# Optional context injection (if you're using FAISS, etc.)
context = "You enjoy journaling and coding. You’re working on being more mindful."

# Create prompt
prompt = f"""<|user|>
{user_input}
<|context|>
{context}
<|assistant|>
"""

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2
    )

# Decode and print reply
reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🤖 Bot Reply:\n", reply.replace(prompt.strip(), "").strip())
